In [1]:
'''
2020-06-04
整理新思路
0、原路径游走
======================
1、word2vec训练+人名机构
2、DBSCAN
3、进行阈值匹配
'''
print(__doc__)


2020-06-04
整理新思路
0、原路径游走
1、word2vec训练+人名机构
2、DBSCAN
3、进行阈值匹配



In [6]:
'''
文本处理
'''
print(__doc__)

import codecs
import json
from os.path import join
import pickle
import os
import re

pubs_raw = json.load(open("train/train_pub.json", 'r', encoding='utf-8'))
pubs_raw1 = json.load(open("sna_data/sna_valid_pub.json", 'r', encoding='utf-8'))

r = '[!“”"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~—～’]+'
f1 = open ('gene/new_text.txt','w',encoding = 'utf-8')

authorname_dict = {}
# 把姓名加进去一起训练
# 先对训练集进行语料库加载
for i,pid in enumerate(pubs_raw):
    pub = pubs_raw[pid]
    
    author_name_new = ""
    org_pstr_t = ""

    # 保存合作者的机构
    for author in pub["authors"]:
        # 保存名字
        if "name" in author:
            authorname = re.sub(r,'', author["name"]).lower()
            taken = authorname.split(" ")
            if len(taken)==2: ##检测目前作者名是否在作者词典中
                authorname = taken[0] + taken[1]
                authorname_reverse = taken[1]  + taken[0] 
            if len(taken)>2:
                authorname = taken[0] + taken[1] + taken[2]
                authorname_reverse = taken[2]  + taken[0] + taken[1]

            # 如果正名字不在字典中
            if authorname not in authorname_dict:
                # 如果反名字也不在字典中
                if authorname_reverse not in authorname_dict:
                    # 则字典的名字置一
                    authorname_dict[authorname]=1
                else:
                    # 否则名字等于反名字
                    authorname = authorname_reverse
            author_name_new = author_name_new + authorname + " "
            
            
            
            
        # 保存机构
        org_pstr = ""
        if "org" in author:
            org = author["org"]
            # 去掉头尾的空格
            org_pstr = org.strip()
            org_pstr = org_pstr.lower()
            # 去除特殊标点
            org_pstr = re.sub(r,' ', org_pstr)
            org_pstr = re.sub(r'\s{2,}', ' ', org_pstr).strip()
            org_pstr_t = org_pstr_t + org_pstr + " "
            
    f1.write(author_name_new+'\n')
    f1.write(org_pstr_t+'\n')

    # 保存文章标题
    title = pub["title"]
    pstr=title.strip()
    pstr = pstr.lower()
    pstr = re.sub(r,' ', pstr)
    pstr = re.sub(r'\s{2,}', ' ', pstr).strip()
    f1.write(pstr+'\n')
    
    # 保存摘要
    if "abstract" in pub and type(pub["abstract"]) is str:
        abstract = pub["abstract"]
        pstr=abstract.strip()
        pstr = pstr.lower()
        pstr = re.sub(r,' ', pstr)
        pstr = re.sub(r'\s{2,}', ' ', pstr).strip()
        f1.write(pstr+'\n')
    
    # 保存机构
    venue = pub["venue"]
    pstr=venue.strip()
    pstr = pstr.lower()
    pstr = re.sub(r,' ', pstr)
    pstr = re.sub(r'\s{2,}', ' ', pstr).strip()
    f1.write(pstr+'\n')
    
# 验证集
for i,pid in enumerate(pubs_raw1):
    pub = pubs_raw1[pid]
    
    author_name_new = ""
    # 保存合作者的机构
    for author in pub["authors"]:
        # 保存名字
        if "name" in author:
            authorname = re.sub(r,'', author["name"]).lower()
            taken = authorname.split(" ")
            if len(taken)==2: ##检测目前作者名是否在作者词典中
                authorname = taken[0] + taken[1]
                authorname_reverse = taken[1]  + taken[0] 
            if len(taken)>2:
                authorname = taken[0] + taken[1] + taken[2]
                authorname_reverse = taken[2]  + taken[0] + taken[1]

            # 如果正名字不在字典中
            if authorname not in authorname_dict:
                # 如果反名字也不在字典中
                if authorname_reverse not in authorname_dict:
                    # 则字典的名字置一
                    authorname_dict[authorname]=1
                else:
                    # 否则名字等于反名字
                    authorname = authorname_reverse
            author_name_new = author_name_new + authorname + " "
    
    org_str_t = ""
    for author in pub["authors"]:
        if "org" in author:
                org = author["org"]
                pstr = org.strip()
                pstr = pstr.lower()
                pstr = re.sub(r,' ', pstr)
                pstr = re.sub(r'\s{2,}', ' ', pstr).strip()
                org_str_t = org_str_t + pstr + " "
    
    f1.write(author_name_new+'\n')
    f1.write(org_str_t+'\n')

        
    title = pub["title"]
    pstr=title.strip()
    pstr = pstr.lower()
    pstr = re.sub(r,' ', pstr)
    pstr = re.sub(r'\s{2,}', ' ', pstr).strip()
    f1.write(pstr+'\n')
    
    if "abstract" in pub and type(pub["abstract"]) is str:
        abstract = pub["abstract"]
        pstr=abstract.strip()
        pstr = pstr.lower()
        pstr = re.sub(r,' ', pstr)
        pstr = re.sub(r'\s{2,}', ' ', pstr).strip()
        f1.write(pstr+'\n')
        
    venue = pub["venue"]
    pstr=venue.strip()
    pstr = pstr.lower()
    pstr = re.sub(r,' ', pstr)
    pstr = re.sub(r'\s{2,}', ' ', pstr).strip()
    f1.write(pstr+'\n')
    
        
f1.close()
print("finish")


文本处理

finish


In [1]:
'''
训练word2vec
'''
print(__doc__)

from gensim.models import word2vec

sentences = word2vec.Text8Corpus(r'gene/new_text.txt')
model = word2vec.Word2Vec(sentences, size=100,negative =5, min_count=2, window=5)
model.save('word2vec/new_word2vec.model')
print("finish")


训练word2vec



In [2]:
'''
离群点匹配
'''
# outlier 离群点集合
def generate_pair(pubs,outlier): ##求匹配相似度
    dirpath = 'gene'
    
    paper_org = {}
    paper_conf = {}
    paper_author = {}
    paper_word = {}
    
    temp=set()
    with open(dirpath + "/paper_org.txt", encoding='utf-8') as pafile:
        for line in pafile:
            temp.add(line)                       
    for line in temp: 
        toks = line.strip().split("\t")
        if len(toks) == 2:
            p, a = toks[0], toks[1]
            if p not in paper_org:
                paper_org[p] = []
            paper_org[p].append(a)
    temp.clear()
    
    with open(dirpath + "/paper_conf.txt", encoding='utf-8') as pafile:
        for line in pafile:
            temp.add(line)                       
    for line in temp: 
        toks = line.strip().split("\t")
        if len(toks) == 2:
            p, a = toks[0], toks[1]
            if p not in paper_conf:
                paper_conf[p]=[]
            paper_conf[p]=a
    temp.clear()
    
    with open(dirpath + "/paper_author.txt", encoding='utf-8') as pafile:
        for line in pafile:
            temp.add(line)                       
    for line in temp: 
        toks = line.strip().split("\t")
        if len(toks) == 2:
            p, a = toks[0], toks[1]
            if p not in paper_author:
                paper_author[p] = []
            paper_author[p].append(a)
    temp.clear()
       
    with open(dirpath + "/paper_word.txt", encoding='utf-8') as pafile:
        for line in pafile:
            temp.add(line)                       
    for line in temp: 
        toks = line.strip().split("\t")
        if len(toks) == 2:
            p, a = toks[0], toks[1]
            if p not in paper_word:
                paper_word[p] = []
            paper_word[p].append(a)
    temp.clear()
    
    
    paper_paper = np.zeros((len(pubs),len(pubs)))
    for i,pid in enumerate(pubs):
        if i not in outlier:
            continue
        for j,pjd in enumerate(pubs):
            if j==i:
                continue
            ca=0
            cv=0
            co=0
            ct=0
          
            if pid in paper_author and pjd in paper_author:
                ca = len(set(paper_author[pid])&set(paper_author[pjd]))*1.5
            if pid in paper_conf and pjd in paper_conf and 'null' not in paper_conf[pid]:
                cv = tanimoto(set(paper_conf[pid]),set(paper_conf[pjd]))
            if pid in paper_org and pjd in paper_org:
                co = tanimoto(set(paper_org[pid]),set(paper_org[pjd]))
            if pid in paper_word and pjd in paper_word:
                ct = len(set(paper_word[pid])&set(paper_word[pjd]))/3
                    
            paper_paper[i][j] =ca+cv+co+ct
            
    return paper_paper


In [10]:
'''
训练获取文章的词向量
并进行聚类
'''
print(__doc__)

import numpy as np
from sklearn.metrics.pairwise import pairwise_distances
from gensim.models import word2vec
from sklearn.cluster import DBSCAN
import json
import re

validate_data = json.load(open("validate_data.json", 'r', encoding='utf-8'))

# 读取word2vec模型
save_model_name = "word2vec/new_word2vec.model"
model_w = word2vec.Word2Vec.load(save_model_name)

# 特殊字符和停用词
r = '[!“”"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~—～’]+'
stopword = ['at','based','in','of','for','on','and','to','an','using','with','the','by','we','be','is','are','can']
stopword1 = ['university','univ','china','department','dept','laboratory','lab','school','al','et',
             'institute','inst','college','chinese','beijing','journal','science','international']

# 姓名字典、最终集
result={}
authorname_dict={}
res_dict = {}
p = []
i=0
j=0

for author_ in validate_data:
    if author_ == 'j_yu':
        result[author_]=[]
        continue
    
    pubs=[]
    # 获取每一个名字聚类里面的论文编号 pub
    for cluster in validate_data[author_]:
        pubs.extend(cluster)
    
    author_copy = author_
    papers = validate_data[author_]
    paper_dict = {}
    # 论文的词向量
    ptext_emb = {}
    author_ = re.sub(r, '', author_).lower()
    print(author_ + " start...")
    
    

    
    # 遍历单人论文
    for pub in papers:
        org=""
        name = ""
        for author in pub["authors"]:
            # 获取name
            authorname = re.sub(r,'', author["name"]).lower()
            taken = authorname.split(" ")

            # 检测作者名称
            # 最终名称为authorname
            if len(taken)==2: 
                authorname = taken[0] + taken[1]
                authorname_reverse = taken[1]  + taken[0] 
            if len(taken)>2:
                authorname = taken[0] + taken[1] + taken[2]
                authorname_reverse = taken[2]  + taken[0] + taken[1]

            # 如果正名字不在字典中
            if authorname not in authorname_dict:
                # 如果反名字也不在字典中
                if authorname_reverse not in authorname_dict:
                    # 则字典的名字置一
                    authorname_dict[authorname]=1
                else:
                    # 否则名字等于反名字
                    authorname = authorname_reverse
            # 不记录待消歧作者
            if authorname == author_:
                continue

            name = name + authorname + " "

            if "org" not in author:
                continue

            a = re.sub(r,'', author["org"]).lower()
            #save org 待消歧作者的机构名
            pstr = a.strip()
            pstr = pstr.lower() #小写
            pstr = re.sub(r,' ', pstr) #去除符号
            pstr = re.sub(r'\s{2,}', ' ', pstr).strip() #去除多余空格
            pstr = pstr.split(' ')
            pstr = [word for word in pstr if len(word)>1]
            pstr = [word for word in pstr if word not in stopword1]
            pstr = [word for word in pstr if word not in stopword]
            pstr=set(pstr)
            temp = ""
            for word in pstr:
                temp = temp + str(word) + " "
            org = org + temp + " "

        #save venue
#         pstr = pub["venue"].strip()
#         pstr = pstr.lower()
#         pstr = re.sub(r,' ', pstr)
#         pstr = re.sub(r'\s{2,}', ' ', pstr).strip()
#         pstr = pstr.split(' ')
#         pstr = [word for word in pstr if len(word)>1]
#         pstr = [word for word in pstr if word not in stopword1]
#         pstr = [word for word in pstr if word not in stopword]
        

        #save text
        pstr = ""    
        keyword=""
        if "keywords" in pub:
            for word in pub["keywords"]:
                keyword = keyword + word + " "
#         pstr = pstr + pub["title"]
#         pstr=pstr.strip()
#         pstr = pstr.lower()
#         pstr = re.sub(r,' ', pstr)
#         pstr = re.sub(r'\s{2,}', ' ', pstr).strip()
#         pstr = pstr.split(' ')
#         pstr = [word for word in pstr if len(word)>1]
#         pstr = [word for word in pstr if word not in stopword]

        #save all words' embedding
        pstr = org + name + keyword + pub["title"] + " " + pub["venue"] 
        
#         if "abstract" in pub:
#             pstr = pstr + " " + str(pub["abstract"]) 
        
        if "year" in pub:
              pstr = pstr +  " " + str(pub["year"])
        
        # 处理
        pstr=pstr.strip()
        pstr = pstr.lower()
        pstr = re.sub(r,' ', pstr)
        pstr = re.sub(r'\s{2,}', ' ', pstr).strip()
        pstr = pstr.split(' ')
        pstr = [word for word in pstr if len(word)>2]
        pstr = [word for word in pstr if word not in stopword]
        pstr = [word for word in pstr if word not in stopword1]

        # 观察第一个
        if j == 0:
            z = ""
            for word in pstr:
                z = z+str(word)+" "
            print(z)
        j += 1
        
        words_vec=[]
        for word in pstr:
            if (word in model_w):
                words_vec.append(model_w[word])
        if len(words_vec)<1:
            words_vec.append(np.zeros(100))
        ptext_emb[str(pub["id"])] = np.mean(words_vec,0)
    
    
    ptext_emb_list = []
    for ptext in ptext_emb:
        ptext_emb_list.append(ptext_emb[ptext])

    if len(ptext_emb_list) == 0:
        ptext_emb_list = np.zeros((2, 2))
    print(len(ptext_emb_list))

    ptext_emb_list = pairwise_distances(ptext_emb_list,metric="cosine")
    print(len(ptext_emb_list))
    temp = np.array(ptext_emb_list)

    print(author_ + " DBSCAN start...")
#     clf = DBSCAN(eps = 0.2, min_samples = 4, metric ="precomputed")
#     s = clf.fit_predict(temp)
    pre = DBSCAN(eps = 0.4, min_samples = 4,metric ="precomputed").fit_predict(temp)
    pre= np.array(pre)

    print(pre)

    
    print("start 离群匹配...")
    outlier=set()
    for i in range(len(pre)):
        if pre[i]==-1:
            outlier.add(i)
            
        
    ##基于阈值的相似性匹配 paper_pair是一个相似度矩阵
    
    
    paper_pair = generate_pair(pubs,outlier)
    # 复制
    paper_pair1 = paper_pair.copy()
    
    # K是种类个数
    K = len(set(pre))
    for i in range(len(pre)):
        if i not in outlier:
            continue
        
        # 找到与当前论文最相近的
        j = np.argmax(paper_pair[i])
        while j in outlier:
            paper_pair[i][j]=-1
            j = np.argmax(paper_pair[i])
        if paper_pair[i][j]>=2:
            pre[i]=pre[j]
        else:
            pre[i]=K
            K=K+1
    
    # 这是利群点间进行匹配
    for ii,i in enumerate(outlier):
        for jj,j in enumerate(outlier):
            if jj<=ii:
                continue
            else:
                if paper_pair1[i][j]>=2:
                    pre[j]=pre[i]
            
    
    # 打印最终结果
    print (pre,len(set(pre)))
      
    # 进行最终分类
    print(author_ + " save start...")
    for label, paper in zip(pre, papers):
        if str(label) not in paper_dict:
            paper_dict[str(label)] = [paper['id']]
        else:
            paper_dict[str(label)].append(paper['id'])
    res_dict[author_copy] = list(paper_dict.values())
    print(str(i) + " finish...")

    i = i+1
#     p.append(ptext_emb)
# path = "gene/_ptext_emb.pkl"    
# json.dump(p, open(path, 'w', encoding='utf-8'), indent=4)
json.dump(res_dict, open('result/new_word2vec_DBSCAN_result_eps_0.2_withoutAbstract_liqunpipei_2.json', 'w', encoding='utf-8'), indent=4)




训练获取文章的词向量
并进行聚类

hengli start...
feifeicheng ywwang martinskitmore pjforsythe petri nets resource management simulation building design modeling resource management building design process information constraint petri nets automation construction 2013 


c:\users\hi\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:162: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
c:\users\hi\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:163: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


768
768
hengli DBSCAN start...
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

c:\users\hi\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:162: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
c:\users\hi\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:163: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).



314
fangchen DBSCAN start...
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
start 离群匹配...
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0

...

[0 0 0 0 1 1 0 0 1 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 1 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1] 2
zhanglei save start...
132 finish...
hywang start...
95
95
hywang DBSCAN start...
[ 0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  0  1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1  1  1  1  1  0  1  1  1
  1  1  1  1  1  1  1  1  0  1  1  1  1  1  1  1  1  1  1  1  1  0  1]
start 离群匹配...
[0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 3
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 4 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1
 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1] 4
hywang save start...
54 finish...
qingli start...


c:\users\hi\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:162: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
c:\users\hi\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:163: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
c:\users\hi\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:162: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
c:\users\hi\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:163: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


1287
1287
qingli DBSCAN start...
[0 0 0 ... 0 0 0]
start 离群匹配...
[0 0 0 ... 0 0 0] 6
qingli save start...
767 finish...
dwang start...


c:\users\hi\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:162: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
c:\users\hi\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:163: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


KeyboardInterrupt: 

In [ ]:
# DBSCAN导入
from sklearn.cluster import DBSCAN
# 对距离计算成对距离模块（不知道啥意思）
from sklearn.metrics.pairwise import pairwise_distances
# 
from sklearn.feature_extraction.text import TfidfVectorizer   # 计算单词权重

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import PCA
# from sklearn.decomposition import TruncatedSVD    
# from sklearn.feature_extraction.text import HasingVectorizer
from sklearn.preprocessing import Normalizer
from sklearn import cluster


res_dict = {}
# 画图
# fig = plt.figure(figsize=(15, 20), dpi=100)
# fig.subplots_adjust(left=.02, right=.98, bottom=.1, top=.96, wspace=.2, hspace=.2)
# plt.title('DBSCAN')
i = 1
# fig.subplots_adjust(hspace=0.4, wspace=0.6)
validate_data = json.load(open("validate_data.json", 'r', encoding='utf-8'))



# 每一个作者信息进行遍历
for author in validate_data:
    
    pubs=[]
    # 获取每一个名字聚类里面的论文编号 pub
    for cluster in validate_data[author_]:
        pubs.extend(cluster)
    
    # print(author)
    coauther_orgs = []
    # 获得论文信息
    papers = validate_data[author]
    # 如果名称下没有内容，放到集合中
    if len(papers) == 0:
        res_dict[author] = []
        continue
    # print(len(papers))
    paper_dict = {}
    
    # 对论文进行遍历
    # 把 姓名+机构+概要
    for paper in papers:
        authors = paper['authors'] 
        names = [precessname(paper_author['name']) for paper_author in authors]
        orgs = [preprocessorg(paper_author['org']) for paper_author in authors if 'org' in paper_author]  
        abstract = paper["abstract"] if 'abstract' in paper else ''
        coauther_orgs.append(etl(' '.join(names + orgs) + ' '+ abstract))        
    # 进行TF-IDF的特征取值
    tfidf = TfidfVectorizer().fit_transform(coauther_orgs)
    
    #使用DBSCAN进行聚类    
    pre = DBSCAN(eps=0.7, min_samples=4, metric='cosine').fit_predict(tfidf)
    
    print(pre)
    
    
    outlier=set()
    for i in range(len(pre)):
        if pre[i]==-1:
            outlier.add(i)
            
        
    ##基于阈值的相似性匹配 paper_pair是一个相似度矩阵
    
    
    paper_pair = generate_pair(pubs,outlier)
    # 复制
    paper_pair1 = paper_pair.copy()
    
    # K是种类个数
    K = len(set(pre))
    for i in range(len(pre)):
        if i not in outlier:
            continue
        
        # 找到与当前论文最相近的
        j = np.argmax(paper_pair[i])
        while j in outlier:
            paper_pair[i][j]=-1
            j = np.argmax(paper_pair[i])
        if paper_pair[i][j]>=1.5:
            pre[i]=pre[j]
        else:
            pre[i]=K
            K=K+1
    
    # 这是利群点间进行匹配
    for ii,i in enumerate(outlier):
        for jj,j in enumerate(outlier):
            if jj<=ii:
                continue
            else:
                if paper_pair1[i][j]>=2:
                    pre[j]=pre[i]
            
    
    # 打印最终结果
    print (pre,len(set(pre)))
      
    # 进行最终分类
    print(author + " save start...")
    for label, paper in zip(pre, papers):
        if str(label) not in paper_dict:
            paper_dict[str(label)] = [paper['id']]
        else:
            paper_dict[str(label)].append(paper['id'])
    res_dict[author] = list(paper_dict.values())
    print(str(i) + " finish...")

    i = i+1    
json.dump(res_dict, open('result/tif_liqun', 'w', encoding='utf-8'), indent=4)
